In [1]:
%env MLFLOW_TRACKING_URI=sqlite:///mlruns.db


env: MLFLOW_TRACKING_URI=sqlite:///mlruns.db


In [11]:
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
import mlflow
mlflow.set_experiment("nyc-taxi-experiment-1")

2024/01/26 19:23:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/01/26 19:23:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/home/jithish/Documents/zoomcamp/04/mlruns/1', creation_time=1706268204714, experiment_id='1', last_update_time=1706268204714, lifecycle_stage='active', name='nyc-taxi-experiment-1', tags={}>

In [6]:
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename) # switched to reading in parquet files

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe(filename="data/green_tripdata_2021-01.parquet")
df_val = read_dataframe(filename="data/green_tripdata_2021-02.parquet")

In [9]:

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
# if models can handle sparse arrays, don't need to do this Jeff says
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.75871521021275

In [15]:
# jeff added pathlib making directory
import pathlib
pathlib.Path("models").mkdir(exist_ok=True)
     

with open('models/lin_reg2.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg2.bin", artifact_path="models_pickle")

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:00:46] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.89072                         
[1]	validation-rmse:9.85084                          
[2]	validation-rmse:9.04239                          
[3]	validation-rmse:8.42145                          
[4]	validation-rmse:7.94876                          
[5]	validation-rmse:7.59260                          
[6]	validation-rmse:7.32418                          
[7]	validation-rmse:7.12473                          
[8]	validation-rmse:6.97518                          
[9]	validation-rmse:6.86206                          
[10]	validation-rmse:6.77695                         
[11]	validation-rmse:6.71082                         
[12]	validation-rmse:6.65974                         
[13]	validation-rmse:6.62065                         
[14]	validation-rmse:6.58845                         
[15]	validation-rmse:6.56261                         
[16]	validation-rmse:6.54201                         
[17]	validation-rmse:6.52355                         
[18]	validation-rmse:6.51093

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:01:55] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.69084                                                   
[1]	validation-rmse:8.26563                                                   
[2]	validation-rmse:7.50020                                                   
[3]	validation-rmse:7.09694                                                   
[4]	validation-rmse:6.87957                                                   
[5]	validation-rmse:6.75500                                                   
[6]	validation-rmse:6.68195                                                   
[7]	validation-rmse:6.63888                                                   
[8]	validation-rmse:6.61236                                                   
[9]	validation-rmse:6.59469                                                   
[10]	validation-rmse:6.58152                                                  
[11]	validation-rmse:6.56955                                                  
[12]	validation-rmse:6.55691                        

...

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:02:50] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.71257                                                  
[1]	validation-rmse:11.25103                                                  
[2]	validation-rmse:10.82647                                                  
[3]	validation-rmse:10.43607                                                  
[4]	validation-rmse:10.07711                                                  
[5]	validation-rmse:9.74910                                                   
[6]	validation-rmse:9.44905                                                   
[7]	validation-rmse:9.17521                                                   
[8]	validation-rmse:8.92316                                                   
[9]	validation-rmse:8.69500                                                   
[10]	validation-rmse:8.48690                                                  
[11]	validation-rmse:8.29763                                                  
[12]	validation-rmse:8.12515                        

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:05:04] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.04154                                                  
[1]	validation-rmse:8.67215                                                   
[2]	validation-rmse:7.84596                                                   
[3]	validation-rmse:7.34421                                                   
[4]	validation-rmse:7.04036                                                   
[5]	validation-rmse:6.87010                                                   
[6]	validation-rmse:6.75927                                                   
[7]	validation-rmse:6.69366                                                   
[8]	validation-rmse:6.64659                                                   
[9]	validation-rmse:6.60780                                                   
[10]	validation-rmse:6.58802                                                  
[11]	validation-rmse:6.57397                                                  
[12]	validation-rmse:6.56358                        

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [24]:
mlflow.xgboost.autolog(disable=True)

In [26]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=25,
        evals=[(valid, 'validation')],
        early_stopping_rounds=3
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:10:53] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390


/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:11:20] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [28]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/01/26 20:21:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/01/26 20:21:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


KeyboardInterrupt: 